## インストール

In [ ]:
pip install pingouin
pip install plotnine
pip install mysql-connector-python
pip install numexpr
pip install python-dotenv
pip install japanize-matplotlib


In [1]:
from itertools import combinations

import csv
import MySQLdb
import mysql.connector as mydb
import pandas as pd
import pandas.io.sql as psql
import numpy as np
from pingouin import kruskal, mwu, multicomp
from plotnine import *


In [78]:
# %load_ext dotenv
# %dotenv

%reload_ext dotenv


In [92]:
host = os.environ.get('HOST')
user = os.environ.get('USER_NAME')
password = os.environ.get('PASSWORD')
database = os.environ.get('DATABASE')
question_database = os.environ.get('QUESTION_UI_DATABASE')
DB_CONFIG = dict(host=host, user=user, password=password, database=database)
QUESTION_DB_CONFIG = dict(
    host=host, user=user, password=password, database=question_database)


def check_db_connection(db_config=QUESTION_DB_CONFIG):
  connection = mydb.connect(**db_config)
  connection.ping(reconnect=True)
  # 接続できているかどうか確認
  print(connection.is_connected())


check_db_connection(QUESTION_DB_CONFIG)


True


In [ ]:
UI_CONDITION = ("question", "perspective", "non")

THEME_CONDITION = ("tpp", "airPollution", "vietnamTrip")

TPP_UI_CONDITION = (
    "TPP_WITH_QUERYUI",
    "TPP_WITH_NONUI",
)
AIRPOLLUTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUERYUI",
    "AIRPOLLUTION_WITH_NONUI",
)
VIETNAMTRIP_UI_CONDITION = (
    "VIETNAMTRIP_WITH_QUERYUI",
    "VIETNAMTRIP_WITH_NONUI"
)


In [91]:

def get_dataframe_from_db(sql, db_config):
  """
  db上のデータをデータフレームとして取得
  """
  conn = mydb.connect(**db_config)
  df_read = pd.read_sql(sql, conn)
  conn.close()
  return df_read


def convert_csv_to_array(csv_path):
  """
  csvファイルの中身を一次元配列へ変換
  """
  result_arr = []
  with open(csv_path) as f:
      reader = csv.reader(f)
      for row in reader:
        result_arr.append(str(row[0]))
  return result_arr


def convert_singleArray_to_dataframe(array, columnName):
  df = pd.DataFrame(array, columns=[columnName],)
  return df


## 分析対象とする実験協力者

- ウェブ検索タスクにて意見文章の提出まで行い，事後アンケートへの回答が完了しているワーカー
- タスク所要時間が外れ値ではないワーカー

In [ ]:

# ウェブ検索タスクと事後アンケートへの回答の両方が完了しているワーカー
questionnaire_answered_crowdIds_csv_path = './data/questionnaire_answered_crowdId.csv'
questionnaire_answered_crowdIds = convert_csv_to_array(questionnaire_answered_crowdIds_csv_path)
questionnaire_answered_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_crowdIds, 'crowdId')
users_who_complete_webSearchTask_df = get_dataframe_from_db("select * from users where hasFinishedWebSearchTask", DB_CONFIG)
users_who_completed_all_task_df =  users_who_complete_webSearchTask_df[users_who_complete_webSearchTask_df['crowdId'].isin(questionnaire_answered_crowdIds)]

all_task_done_user_crowdIds = users_who_completed_all_task_df['crowdId'].tolist()

# タスク所要時間の外れ値処理
def get_dataframe_without_outlier(df, target_column):
    q1 = df[target_column].quantile(
        0.25)
    q3 = df[target_column].quantile(
        0.75)
    iqr = q3 - q1
    threshold_upper = q3 + 1.5 * iqr
    threshold_lower = q1 - 1.5 * iqr
    df["threshold_lower"] = threshold_lower
    df["threshold_upper"] = threshold_upper

    df_without_outlier = df[(df[target_column] < threshold_upper) & (
        df[target_column] > threshold_lower)]
    return df_without_outlier

task_time_df = get_dataframe_from_db("select * from taskTime;", DB_CONFIG)
task_time_with_users_done_all_task_df = task_time_df[task_time_df['crowdId'].isin(
    all_task_done_user_crowdIds)]
task_time_with_users_done_all_task_df["taskTimeDuration"] = (
    task_time_with_users_done_all_task_df["finished_at"]-task_time_with_users_done_all_task_df["reStarted_at"]).map(lambda x: x.total_seconds())

valid_users_df = get_dataframe_without_outlier(
    task_time_with_users_done_all_task_df, 'taskTimeDuration')
valid_users_df.head()
valid_user_crowdIds = valid_users_df.crowdId.tolist()
print(len(valid_user_crowdIds))
# print(len(valid_users_df)) # 264人

## 問いかけ群ワーカー
# ウェブ検索タスクと事後アンケートへの回答の両方が完了している問いかけ群ワーカー
questionnaire_answered_questionUI_crowdIds_csv_path = './data/questionnaire_answered_questionUI_crowdId.csv'
questionnaire_answered_questionUI_crowdIds = convert_csv_to_array(
    questionnaire_answered_questionUI_crowdIds_csv_path)
questionnaire_answered_questionUI_crowdIds_df = convert_singleArray_to_dataframe(
    questionnaire_answered_questionUI_crowdIds, 'crowdId')
q_users_df = get_dataframe_from_db(
    "select * from users where hasFinishedWebSearchTask;", QUESTION_DB_CONFIG)
q_users_who_completed_all_task_df = q_users_df[q_users_df['crowdId'].isin(
    questionnaire_answered_questionUI_crowdIds)]
q_all_task_done_user_crowdIds = q_users_who_completed_all_task_df['crowdId'].tolist()

q_task_time_df = get_dataframe_from_db('select * from taskTime;', QUESTION_DB_CONFIG)
q_task_time_with_users_done_all_task_df = q_task_time_df[q_task_time_df['crowdId'].isin(
    q_all_task_done_user_crowdIds)]
q_task_time_with_users_done_all_task_df["taskTimeDuration"] = (
    q_task_time_with_users_done_all_task_df["finished_at"]-q_task_time_with_users_done_all_task_df["reStarted_at"]).map(lambda x: x.total_seconds())
q_valid_users_df = get_dataframe_without_outlier(
    q_task_time_with_users_done_all_task_df, 'taskTimeDuration')
q_valid_user_crowdIds = q_valid_users_df.crowdId.tolist()
print(len(q_valid_user_crowdIds)) # 87人




## 実験協力者の内訳

- questionUI群に関しては，別データで分析を行うため，perspectiveUI，nonUIのみ集計する

In [107]:
THEME_UI_CONDITION = (
                        "TPP_WITH_QUERYUI",
                        "AIRPOLLUTION_WITH_QUERYUI",
                        "VIETNAMTRIP_WITH_QUERYUI",
                        "TPP_WITH_NONUI",
                        "AIRPOLLUTION_WITH_NONUI",
                        "VIETNAMTRIP_WITH_NONUI"
                        )

users_df = get_dataframe_from_db('select * from users;', DB_CONFIG)
users_df = users_df[users_df['crowdId'].isin(
    valid_user_crowdIds)]

for theme_ui_condition in THEME_UI_CONDITION:
    res = users_df.query('taskId == @theme_ui_condition')

# 問いかけ群ワーカー
QUESTION_UI_CONDITION = (
    "TPP_WITH_QUESTIONUI",
    "AIRPOLLUTION_WITH_QUESTIONUI",
    "VIETNAMTRIP_WITH_QUESTIONUI"
)

q_users_df = get_dataframe_from_db("select * from users;", QUESTION_DB_CONFIG)
q_users_df = q_users_df[q_users_df['crowdId'].isin(q_valid_user_crowdIds)]



for condition in QUESTION_UI_CONDITION:
    res = q_users_df.query('taskId == @condition')
    print(len(res))

question_UI_participant = [27, 32, 28]
perspective_UI_participant = [32, 30, 25]
non_UI_participant = [33, 35, 30]

participant_list = [
    question_UI_participant,
    perspective_UI_participant,
    non_UI_participant 
]

participant_df = pd.DataFrame(participant_list)
participant_df.index = ['questionUI','perspectiveUI', 'nonUI']
participant_df.columns = ['TPP', '大気汚染', 'ベトナム旅行']
participant_df.head()





27
32
28


<ipython-input-91-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
<ipython-input-91-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


In [93]:
res = get_dataframe_from_db('select * from users;', QUESTION_DB_CONFIG)
res.head()


<ipython-input-91-afed83266c14>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,id,crowdId,password,taskId,hasFinishedWebSearchTask,created_at,updated_at
0,1,tatituteto013,"$argon2id$v=19$m=65536,t=3,p=4$z35FSk7WAzCTxoa...",TPP_WITH_QUESTIONUI,0,2023-01-13 10:30:45,2023-01-13 10:30:45
1,2,k1030,"$argon2id$v=19$m=65536,t=3,p=4$UhnBSGGzVi4mdlx...",AIRPOLLUTION_WITH_QUESTIONUI,1,2023-01-13 10:31:03,2023-01-13 10:53:28
2,3,higuchika,"$argon2id$v=19$m=65536,t=3,p=4$clr1F/X9k9CyLFC...",VIETNAMTRIP_WITH_QUESTIONUI,1,2023-01-13 10:31:05,2023-01-13 10:43:18
3,4,ri-chan,"$argon2id$v=19$m=65536,t=3,p=4$fPwXIjRAhGkyhfw...",TPP_WITH_QUESTIONUI,1,2023-01-13 10:31:09,2023-01-13 10:53:10
4,5,lohizuka,"$argon2id$v=19$m=65536,t=3,p=4$cgdPqGxrU82vPc6...",AIRPOLLUTION_WITH_QUESTIONUI,1,2023-01-13 10:32:03,2023-01-13 11:05:51


In [62]:
def get_dataframe_without_invalid_user(df):
  return df[df['crowdId'].isin(valid_user_crowdIds)]


# クエリ発行回数
query_count_sql = """
select
  users.crowdId, users.taskId, count(users.crowdId) as query_count
from
  queryLogs as ql
inner join
  users on ql.crowdId = users.crowdId
group by
  users.crowdId, users.taskId
"""

query_count_df = get_dataframe_from_db(query_count_sql)
query_count_df_without_invalid_user = get_dataframe_without_invalid_user(query_count_df)
query_count_df_without_invalid_user.head()

TPP_UI_CONDITION = (
    "TPP_WITH_QUERYUI",
    "TPP_WITH_NONUI",
)
AIRPOLLUTION_UI_CONDITION = (
    "AIRPOLLUTION_WITH_QUERYUI",
    "AIRPOLLUTION_WITH_NONUI",
)
VIETNAMTRIP_UI_CONDITION = (
    "VIETNAMTRIP_WITH_QUERYUI",
    "VIETNAMTRIP_WITH_NONUI"
)

for condition in TPP_UI_CONDITION:
  target_df = query_count_df_without_invalid_user.query('taskId == @condition')
  print(condition)
  print('中央値',target_df["query_count"].median())
  print('標準偏差',target_df["query_count"].std())

for condition in AIRPOLLUTION_UI_CONDITION:
  target_df = query_count_df_without_invalid_user.query('taskId == @condition')
  print(condition)
  print('中央値',target_df["query_count"].median())
  print('標準偏差',target_df["query_count"].std())

for condition in VIETNAMTRIP_UI_CONDITION:
  target_df = query_count_df_without_invalid_user.query('taskId == @condition')
  print(condition)
  print('中央値', target_df["query_count"].median())
  print('標準偏差',target_df["query_count"].std())


query_count_df_without_invalid_user.head()



TPP_WITH_QUERYUI
中央値 1.0
標準偏差 1.157026221578126
TPP_WITH_NONUI
中央値 1.0
標準偏差 2.0757437513364025
AIRPOLLUTION_WITH_QUERYUI
中央値 1.0
標準偏差 1.4064710873459967
AIRPOLLUTION_WITH_NONUI
中央値 1.0
標準偏差 1.1725143564009723
VIETNAMTRIP_WITH_QUERYUI
中央値 1.0
標準偏差 3.0506829836393465
VIETNAMTRIP_WITH_NONUI
中央値 1.0
標準偏差 1.9666764075348089


<ipython-input-11-751477a6ed0c>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


,crowdId,taskId,query_count
0,o1571139,VIETNAMTRIP_WITH_NONUI,1
1,bvglp11,TPP_WITH_NONUI,1
2,takahiro0655,TPP_WITH_QUERYUI,1
3,a_11044,AIRPOLLUTION_WITH_QUESTIONUI,7
4,Hiroppi1,AIRPOLLUTION_WITH_QUERYUI,4
